In [16]:
# import dependencies
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import loompy as lp
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

from IPython.display import HTML, display

In [2]:
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=120)

-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                         9.2.0
asttokens                   NA
attr                        22.1.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
boltons                     NA
bottleneck                  1.3.5
cloudpickle                 2.2.0
ctxcore                     0.2.0
cycler                      0.10.0
cython_runtime              NA
cytoolz                     0.12.0
dask                        2022.9.1
dateutil                    2.8.2
debugpy                     1.5.1
decorator                   5.1.1
entrypoints                 0.4
executing                   0.8.3
frozendict                  2.3.4
fsspec                      2022.8.2
h5py                        3.7.0
ipykernel                   6.15.2
jedi                        0.18.1
jinja2                      3.1.2
joblib                      1.2.0
kiwisolver                  1.4.2
llvmlite                    0.39.1
loomp

In [3]:
# Set maximum number of jobs for Scanpy.
sc.settings.njobs = 32

In [4]:
RESOURCES_FOLDERNAME = "/home/shpc_100839/scRNA_BC_metastases/Data/integrate_sc_BC/tumor/h5ad/filter_NG/"
AUXILLIARIES_FOLDERNAME = "/home/shpc_100839/software/SCENICdata/"
RESULTS_FOLDERNAME = "/home/shpc_100839/scRNA_BC_metastases/Data/integrate_sc_BC/tumor/SCENIC/"
FIGURES_FOLDERNAME = "/home/shpc_100839/scRNA_BC_metastases/Data/integrate_sc_BC/tumor/SCENIC/"

In [5]:
sc.settings.figdir = FIGURES_FOLDERNAME

In [6]:
BASE_URL = "http://motifcollections.aertslab.org/v10nr_clust/logos/"
COLUMN_NAME_LOGO = "MotifLogo"
COLUMN_NAME_MOTIF_ID = "MotifID"
COLUMN_NAME_TARGETS = "TargetGenes"

In [7]:
def savesvg(fname: str, fig, folder: str=FIGURES_FOLDERNAME) -> None:
    """
    Save figure as vector-based SVG image format.
    """
    fig.tight_layout()
    fig.savefig(os.path.join(folder, fname), format='svg')

In [8]:
def display_logos(df: pd.DataFrame, top_target_genes: int = 3, base_url: str = BASE_URL):
    """
    :param df:
    :param base_url:
    """
    # Make sure the original dataframe is not altered.
    df = df.copy()
    
    # Add column with URLs to sequence logo.
    def create_url(motif_id):
        return '<img src="{}{}.png" style="max-height:124px;"></img>'.format(base_url, motif_id)
    df[("Enrichment", COLUMN_NAME_LOGO)] = list(map(create_url, df.index.get_level_values(COLUMN_NAME_MOTIF_ID)))
    
    # Truncate TargetGenes.
    def truncate(col_val):
        return sorted(col_val, key=op.itemgetter(1))[:top_target_genes]
    df[("Enrichment", COLUMN_NAME_TARGETS)] = list(map(truncate, df[("Enrichment", COLUMN_NAME_TARGETS)]))
    
    MAX_COL_WIDTH = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', -1)
    display(HTML(df.head().to_html(escape=False)))
    pd.set_option('display.max_colwidth', MAX_COL_WIDTH)

In [9]:
HUMAN_TFS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'allTFs_hg38.txt')
RANKING_DBS_FNAMES = os.path.join(AUXILLIARIES_FOLDERNAME,'hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather')
                                                        # hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
MOTIF_ANNOTATIONS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl')

In [10]:
SAMPLE_ID = "atac"

In [11]:
EXP_H5AD_FNAME = os.path.join(RESOURCES_FOLDERNAME, 'atac.h5ad')

In [12]:
EXP_FILTER_LOOM_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.h5ad.filter.loom'.format(SAMPLE_ID))
ADJACENCIES_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.adjacencies.tsv'.format(SAMPLE_ID))
MOTIFS_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.motifs.csv'.format(SAMPLE_ID))
REGULONS_DAT_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.regulons.dat'.format(SAMPLE_ID))
AUCELL_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.auc.csv'.format(SAMPLE_ID))
BIN_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.bin.csv'.format(SAMPLE_ID))
THR_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.thresholds.csv'.format(SAMPLE_ID))
ANNDATA_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.h5ad'.format(SAMPLE_ID))
LOOM_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.loom'.format(SAMPLE_ID))

In [35]:
adata = sc.read_h5ad(EXP_H5AD_FNAME)
adata

AnnData object with n_obs × n_vars = 90 × 8872
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'cell.id', 'singler.label', 'singler.celltype.compartment', 'paper.celltype.compartment', 'paper.celltype.major', 'paper.celltype.minor', 'paper.celltype.subset', 'primary.subtype', 'site', 'study', 'celltype.compartment', 'sample', 'cell.names', 'copykat.pred'
    var: 'features'

## filtering

In [14]:
nCountsPerGene = np.sum(adata.X, axis=0)
nCellsPerGene = np.sum(adata.X>0, axis=0)
print("Number of counts (in the dataset units) per gene:", nCountsPerGene.min(), " - " ,nCountsPerGene.max())
print("Number of cells in which each gene is detected:", nCellsPerGene.min(), " - " ,nCellsPerGene.max())

Number of counts (in the dataset units) per gene: 5.0  -  49037.0
Number of cells in which each gene is detected: 5  -  90


In [17]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( EXP_FILTER_LOOM_FNAME, adata.X.transpose(), row_attrs, col_attrs)


### Step1 Network inference based on GRNBoost2

In [36]:
!pyscenic grn {EXP_FILTER_LOOM_FNAME} {HUMAN_TFS_FNAME} \
    -o {ADJACENCIES_FNAME} \
    --num_workers 8 \
    --method grnboost2


2022-09-19 17:36:13,416 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2022-09-19 17:36:13,517 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
preparing dask client
parsing input
creating dask graph
16 partitions
computing dask graph
not shutting down client, client was created externally
finished
2022-09-19 17:56:06,827 - distributed.worker - WARNING - Heartbeat to scheduler failed
Traceback (most recent call last):
  File "/home/shpc_100839/miniconda3/envs/pyscenic/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/shpc_100839/miniconda3/envs/pyscenic/lib/python3.10/site-packages/distributed/worker.py", line 1191, in heartbeat
    response = await retry_operation(
  File "/home/shpc_100839/miniconda3/envs/pys

### STEP 2-3: Regulon prediction aka cisTarget from CLI

In [24]:
DBS_PARAM = RANKING_DBS_FNAMES
!pyscenic ctx {ADJACENCIES_FNAME} {DBS_PARAM} --annotations_fname {MOTIF_ANNOTATIONS_FNAME} --expression_mtx_fname {EXP_FILTER_LOOM_FNAME} --output {MOTIFS_FNAME} --num_workers 4


2022-09-19 17:15:31,024 - pyscenic.cli.pyscenic - INFO - Creating modules.

2022-09-19 17:15:31,028 - pyscenic.cli.pyscenic - ERROR - No columns to parse from file


In [ ]:
df_motifs = load_motifs(MOTIFS_FNAME)
df_motifs.head()

In [ ]:
display_logos(df_motifs.head())

### STEP 4: Cellular enrichment aka AUCell

In [ ]:
pyscenic.transform.df2regulons

In [ ]:
!pyscenic aucell {EXP_FILTER_LOOM_FNAME} {MOTIFS_FNAME} --output sample_SCENIC.loom --num_workers 8
